In [87]:
import pandas as pd
from datetime import datetime
import io
import numpy as np


In [88]:
def process_date(date_string_series):
    formatted_date_series = pd.to_datetime(date_string_series, format="%m/%d/%Y").dt.strftime("%Y年%m月%d日")
    return formatted_date_series

In [89]:
df_input = pd.read_csv('accountactivity-18.csv')

In [90]:
columns_exp = ['日期', '类型', '金额', '一级分类', '二级分类', '账户1', '账户2', '备注', '货币', '标签']
df_exp = pd.DataFrame(columns=columns_exp)

In [91]:
columns_TD = ["Date", "Description", "Amount_1", "Amount_2", "Balance"]
df_input.columns = columns_TD

In [ ]:
# account_mapping = {"TD CASH BACK VISA* CARD":308, "TD EVERY DAY SAVINGS ACCOUNT":995, "TD UNLIMITED CHEQUING ACCOUNT":995 }
# transaction_mapping = {"Income":"收入", "Interest Income":"收入",}
category_mapping ={   
'FRESHCO': ['Grocery'],
'APPLE': ['数码', '软件'],
'LUCY':['餐饮', 'dine in'],
'SHOPPERS DRUG':['购物'],
'MCDONALD':['餐饮', 'dine in'],
'MARSHALLS/HOMES':['购物','日用'],
'MEAL':['餐饮', 'dine in'],
'FIDO MOBILE':['Phone Bill'],
'WESTERN DRUG MART':['购物'],
'IVEY BUSINESS SCHOOL': ['餐饮', 'dine in'],
'SEPHORA': ['护肤'],
'Temu': ['购物'],
'URBAN PLANET':['购物'],
'COCA COLA': ['餐饮','零食'],
'OPENAI':['数码','应用软件'],
'WHOLE FOODS':['Grocery'],
'EUREST':['餐饮', 'work'],
'XIANG ZI':['餐饮', 'dine in'],
'APPLE.COM':['数码','应用软件'],
'GOOD CATCH BAR': ['餐饮', 'dine in'],
'UBER':['交通','打车'],
'PROVIDENT ENERG':['住房', '水电费'],
'SQUARE ONE INS':['住房', '住房保险'],
'AMZN Mktp CA':['购物']
}
df_exp['一级分类'] = df_input['Description'].apply(lambda x: next((category_mapping[key][0] for key in category_mapping.keys() if key in x), None))
df_exp['二级分类'] = df_input['Description'].apply(lambda x: next((category_mapping[key][1] for key in category_mapping.keys() if key in x and len(category_mapping[key]) > 1), None))

In [93]:


# This covers both actual empty strings and null/NaN values
df_exp['一级分类'] = df_exp['一级分类'].replace(['', np.nan, None], '餐饮')

In [94]:
df_exp

,日期,类型,金额,一级分类,二级分类,账户1,账户2,备注,货币,标签
0,NaN,NaN,NaN,数码,应用软件,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Grocery,None,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,餐饮,work,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,餐饮,work,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,餐饮,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
62,NaN,NaN,NaN,交通,打车,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,餐饮,None,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,交通,打车,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,交通,打车,NaN,NaN,NaN,NaN,NaN


In [95]:
# df_exp['日期'] = process_date(df_input['Date'])
df_input['Date'] = pd.to_datetime(df_input['Date'])
df_exp['日期'] = df_input['Date'].dt.strftime('%Y年%m月%d日')

In [96]:
df_exp

,日期,类型,金额,一级分类,二级分类,账户1,账户2,备注,货币,标签
0,2025年11月03日,NaN,NaN,数码,应用软件,NaN,NaN,NaN,NaN,NaN
1,2025年11月03日,NaN,NaN,Grocery,None,NaN,NaN,NaN,NaN,NaN
2,2025年11月04日,NaN,NaN,餐饮,work,NaN,NaN,NaN,NaN,NaN
3,2025年11月06日,NaN,NaN,餐饮,work,NaN,NaN,NaN,NaN,NaN
4,2025年11月07日,NaN,NaN,餐饮,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
62,2026年01月06日,NaN,NaN,交通,打车,NaN,NaN,NaN,NaN,NaN
63,2026年01月06日,NaN,NaN,餐饮,None,NaN,NaN,NaN,NaN,NaN
64,2026年01月07日,NaN,NaN,交通,打车,NaN,NaN,NaN,NaN,NaN
65,2026年01月07日,NaN,NaN,交通,打车,NaN,NaN,NaN,NaN,NaN


In [97]:
# Initialize columns
df_exp['金额'] = df_input['Amount_2']
df_exp['类型'] = "收入"

# Update rows where Amount_1 has data
mask = (df_input['Amount_1'].fillna('') != '')
df_exp.loc[mask, '金额'] = df_input['Amount_1']
df_exp.loc[mask, '类型'] = "支出"

In [98]:
df_exp['备注'] = df_input['Description']
# df_exp['金额'] = df_input['Amount']
# df_exp['类型'] = df_input['Category'].map(transaction_mapping)
# df_exp['类型'].fillna("支出", inplace=True)
# df_exp['账户1'] = df_input['Account Name'].map(account_mapping)

df_exp['货币'] = 'CAD'


In [99]:
df_exp.to_csv('export.csv', index=False)

In [100]:
df_exp

,日期,类型,金额,一级分类,二级分类,账户1,账户2,备注,货币,标签
0,2025年11月03日,支出,32.84,数码,应用软件,NaN,NaN,OPENAI *CH 22.60_V,CAD,NaN
1,2025年11月03日,支出,64.35,Grocery,None,NaN,NaN,WHOLE FOODS MAR _M,CAD,NaN
2,2025年11月04日,支出,14.09,餐饮,work,NaN,NaN,EUREST-23511 _M,CAD,NaN
3,2025年11月06日,支出,15.27,餐饮,work,NaN,NaN,EUREST-23511 _M,CAD,NaN
4,2025年11月07日,收入,2934.32,餐饮,None,NaN,NaN,IBM Canada Ltd. PAY,CAD,NaN
...,...,...,...,...,...,...,...,...,...,...
62,2026年01月06日,支出,91.56,交通,打车,NaN,NaN,UBER CANADA/UBE _V,CAD,NaN
63,2026年01月06日,收入,1.09,餐饮,None,NaN,NaN,UberDirectCA_PA _V,CAD,NaN
64,2026年01月07日,收入,91.56,交通,打车,NaN,NaN,UBER CAN REV 01/06_V,CAD,NaN
65,2026年01月07日,支出,68.78,交通,打车,NaN,NaN,UBER CANADA/UBE _V,CAD,NaN
